This notebook shows the reconstruction of a 3D GRE with two almost identical echotimes sampled with 3D Lissajous variable density shell trajectories.


In [10]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

import sys

sys.path.insert(0, "../src")

import h5py
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import torch

from juart.conopt.functional.fourier import nonuniform_fourier_transform_adjoint
from juart.conopt.tfs.fourier import nonuniform_transfer_function
from juart.recon.sense import SENSE

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load preprocessed dataset

In [11]:
data_path = "data/3DLiss_vd_preproc.h5"
with h5py.File(data_path, "r") as f:
    print(f"Dataset holds following data: {f.keys()}")

    print(f"Coilsensitivity info: {f['coilsens'].attrs['info']}")
    print(f"Trajectory info: {f['k'].attrs['info']}")
    print(f"Signal info: {f['d'].attrs['info']}")

    shape = (156, 156, 156, 2)
    ktraj = f["k"][:]
    coilsens = f["coilsens"][:]
    d = f["d"][:]

    print(f"Coilsensitivity shape {coilsens.shape}")
    print(f"Trajectory shape {ktraj.shape}")
    print(f"Signal shape {d.shape}")

Dataset holds following data: <KeysViewHDF5 ['coilsens', 'd', 'k']>
Coilsensitivity info: Shape (Channels, Nx, Ny, Nz).
Trajectory info: Shape (Dimensions, Samples, Echotimes). Scaled in units of cycle/fov
Signal info: Shape (Channels, Samples, Echotimes).
Coilsensitivity shape (8, 156, 156, 156)
Trajectory shape (3, 2001191, 2)
Signal shape (8, 2001191, 2)


# Convert data to JUART format

In [12]:
# Scale trajectory to [-0.5, 0.5]
print(f"Min/Max of trajectory: {ktraj.min()} / {ktraj.max()}")
ktraj = ktraj / (2 * ktraj.max())
print(f"Min/Max of trajectory: {ktraj.min()} / {ktraj.max()}")

ktraj = torch.from_numpy(ktraj)
coilsens = torch.from_numpy(coilsens)
d = torch.from_numpy(d)

Min/Max of trajectory: -78.62955474853516 / 78.626708984375
Min/Max of trajectory: -0.5000181198120117 / 0.5


# Perform 3D CG-SENSE reconstruction

In [15]:
AHd = nonuniform_fourier_transform_adjoint(ktraj, d, (156, 156, 156))
AHd = torch.sum(torch.conj(coilsens[..., None]) * AHd, dim=0)

print(AHd.shape)

torch.Size([156, 156, 156, 2])


In [5]:
H = nonuniform_transfer_function(ktraj, (1, 156, 156, 156, 2), oversampling=(2, 2, 2))

In [6]:
cg_solver = SENSE(coilsens, AHd, H, axes=(1, 2, 3), maxiter=10, verbose=True)

In [7]:
cg_image = cg_solver.solve().view(torch.complex64).reshape(shape)

[CG] Iter: 09 Res: 1.29E-01 : 100%|██████████| 10/10 [01:30<00:00,  9.04s/it]


In [8]:
def plot_image(z):
    vmax = cg_image[:, :, z].abs().max()
    plt.imshow(np.abs(cg_image[:, :, z, 1]), cmap="gray", vmax=vmax)
    plt.show()


Slider = widgets.IntSlider(
    value=80,
    min=0,
    max=155,
    step=1,
)

plt.figure(figsize=(10, 5))
plt.title("CG-SENSE Recon")

widgets.interact(plot_image, z=(0, 155, 1))

interactive(children=(IntSlider(value=77, description='z', max=155), Output()), _dom_classes=('widget-interact…

<function __main__.plot_image(z)>